In [ ]:
# 1.

# ## RAG 구성 해서 사용자의 질문이 들어오면 문서를 기반으로 검색을 하고 LLM에게 전달하는 것을 구성해보자.
#  - retrieve를 해줄 노드를 만들어보자.
#  - document를 읽어와서 적당한 사이즈로 잘라주는 작업이 필요하다.

# ## 라이브러리 설치
#  - pypdf : pdf 파일을 읽어오기 위한 라이브러리
#  - langchain-community : langchain 라이브러리의 커뮤니티 라이브러리
#  - langchain-text-splitters : 문서를 잘라주는 라이브러리

%pip install -qU pypdf langchain-community langchain-text-splitters


In [2]:
# 문서를 읽어오는 노드를 만들어보자. / langchain_community 공식 문서의 예제를 그대로 사용해보자.

from langchain_community.document_loaders import PyPDFLoader
pdf_file_path = './income_tax.pdf'
loader = PyPDFLoader(pdf_file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
# pages를 출력해보자. 쪽수별로 쪼개 놓았다.
# pages

# 55조를 출력해보자.
pages[35]

In [ ]:
# 2.

# # 표가 없이 다음으로 넘어가 버린다. ※langchain에서 기본으로 제공하는 PDF로더는 PDF 파일 안의 이미지를 파싱하지 못한다. 
#  - 따라서 이미지를 보고 싶으면 다른 라이브러리를 설치 해야 한다. 
#  - 다른 방법은 chat-gpt를 활용해서 "제 55조의 테이블을 파싱해주세요" 라고 질문을 해서 테이블을 파싱해주는 방법이 있다.
#  - 그런데 모든 문서에 대해서 일일이 전처리를 할 수 없다. 따라서 이미지를 파싱할 수 있는 라이브러리를 설치해야 한다.
#  - 강사가 추천하는 라이브러리는 파이썬 패키지 중 하나인 zerox이다. LLM을 활용해서 OCR을 돌려서 문서를 인식하는 패키지이다.
#  - github.com/getomni-ai/zerox
%pip install -q py-zerox

In [ ]:
# 아래 gemini 모델을 사용하기 위해서 환경변수를 불러와야 한다.

from dotenv import load_dotenv
load_dotenv()

In [ ]:
# 아래 코드를 그냥 돌리면 에러가 난다. asyncio를 돌릴 때 이벤트루프가 없어야 하는데 notebook이 디폴트로뭔가 돌리는게 있어서 에러가 난다다.
# 따라서 아래 코드를 돌리기 위해서는 패키지를 설치해야 한다.

%pip install -q nest_asyncio



In [7]:
import nest_asyncio
nest_asyncio.apply()



In [ ]:
# 2.1

# ## 추가 설치 해야 할 패키지(강사는 안함: 애플은 필요 없는 패키지 같음)
# # Poppler 설치: Poppler(https://github.com/oschwartz10612/poppler-windows/releases/download/v24.08.0-0/Release-24.08.0-0.zip)를 다운로드하여 설치합니다. 운영체제에 맞는 Poppler 바이너리를 다운로드하여 압축을 풀고 적절한 위치에 저장합니다. 
#  - (Windows의 경우, bin 폴더의 경로를 기억해두세요.)
# # 환경 변수 설정 (Windows): (1) 시스템 환경 변수 편집기(검색창에 "환경 변수" 검색)를 엽니다.
# #                          (2) "시스템 속성" 창에서 "환경 변수" 버튼을 클릭합니다.
# #                          (3) "시스템 변수" 섹션에서 "Path" 변수를 선택하고 "편집" 버튼을 클릭합니다.
# #                          (4) "새로 만들기" 버튼을 클릭하고 Poppler bin 폴더의 경로를 추가합니다. (예: C:\path\to\poppler-x.xx.x\bin)
# #                          (5) 모든 창을 닫고 변경 사항을 저장합니다.
# #                          (6) 터미널 또는 IDE 재시작: 환경 변수 변경 사항이 적용되도록 터미널 또는 IDE를 재시작합니다.

# 예제는 github에서 그대로 복사해 오면 된다.
# LightLLM 이라는 래퍼를 사용해서 OCR을 돌리기 때문에 모델 이름과 환경변수를 주면 상용 LLM을 쓸 수 있다.
# 여기서는 google-gemini-1.5-flash-001 이라는 모델을 사용하므로 다른 코드들은 삭제한다.

from pyzerox import zerox
import os
import json
import asyncio

### Model Setup (Use only Vision Models) Refer: https://docs.litellm.ai/docs/providers ###

## placeholder for additional model kwargs which might be required for some models
kwargs = {}

## system prompt to use for the vision model
custom_system_prompt = None

# to override
# custom_system_prompt = "For the below pdf page, do something..something..." ## example

# ###################### Example for OpenAI ######################
# model = "gpt-4o-mini" ## openai model
# os.environ["OPENAI_API_KEY"] = "" ## your-api-key


# ###################### Example for Azure OpenAI ######################
# model = "azure/gpt-4o-mini" ## "azure/<your_deployment_name>" -> format <provider>/<model>
# os.environ["AZURE_API_KEY"] = "" # "your-azure-api-key"
# os.environ["AZURE_API_BASE"] = "" # "https://example-endpoint.openai.azure.com"
# os.environ["AZURE_API_VERSION"] = "" # "2023-05-15"


###################### Example for Gemini ######################
model = "gemini/gemini-2.0-flash-exp" ## "gemini/<gemini_model>" -> format <provider>/<model>
# os.environ['GEMINI_API_KEY'] = "" # your-gemini-api-key  : 환경변수는 따로 불러오면 되므로 여기선 안쓴다.


# ###################### Example for Anthropic ######################
# model="claude-3-opus-20240229"
# os.environ["ANTHROPIC_API_KEY"] = "" # your-anthropic-api-key

# ###################### Vertex ai ######################
# model = "vertex_ai/gemini-1.5-flash-001" ## "vertex_ai/<model_name>" -> format <provider>/<model>
# ## GET CREDENTIALS
# ## RUN ##
# # !gcloud auth application-default login - run this to add vertex credentials to your env
# ## OR ##
# file_path = 'path/to/vertex_ai_service_account.json'

# # Load the JSON file
# with open(file_path, 'r') as file:
#     vertex_credentials = json.load(file)

# # Convert to JSON string
# vertex_credentials_json = json.dumps(vertex_credentials)

# vertex_credentials=vertex_credentials_json

# ## extra args
# kwargs = {"vertex_credentials": vertex_credentials}

# ###################### For other providers refer: https://docs.litellm.ai/docs/providers ######################

# Define main async entrypoint
async def main():
    file_path = "./pixel-nvidia.pdf" ## local filepath and file URL supported

    ## process only some pages or all
    select_pages = None ## None for all, but could be int or list(int) page numbers (1 indexed)

    output_dir = "./documents" ## directory to save the consolidated markdown file
    result = await zerox(file_path=file_path, model=model, output_dir=output_dir,
                        custom_system_prompt=custom_system_prompt,select_pages=select_pages, **kwargs)
    return result


# run the main function:
result = asyncio.run(main())

# print markdown result
print(result)

In [ ]:
# 3.
## 마크다운이 생겼으니 랭체인이 제공하는 마크다운 로더를 사용해서 문서를 읽어오자.

%pip install -q "unstructured[md]" nltk

In [10]:
# 3.1

## pdf loader와 차이점 : pdf는 pages로 큰 문서를 알아서 쪼갰다. 페이지 단위로.
## 마크다운은 페이지 단위로 쪼개지 않고 통으로 읽어온다. "data = loader.load()" 이렇게 하면 문서가 하나로 읽힌다.
## 이럴 때는 langchain의 text splitter를 사용해서 쪼개줘야 한다.(맨 처음에 깔아줬던 패키지)

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500, 
    chunk_overlap = 100,
    separators = ["\n\n", "\n"]
)

# 이렇게 해주고 그 다음 load 대신에 load_and_split 함수를 사용한다.

In [11]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

markdown_path = "./documents/income_tax.md"
loader = UnstructuredMarkdownLoader(markdown_path)
# data = loader.load()
# assert len(data) == 1
# assert isinstance(data[0], Document)
# readme_content = data[0].page_content
# print(readme_content[:250])
document_list = loader.load_and_split(text_splitter)



In [ ]:
document_list[43]




In [ ]:
# 4.

# 이렇게 해주면 문서가 쪼개져서 리스트로 나온다. 그런데 표가 잘려서 안 보인다. 정확한 컨텍스트를 전달하는건 아니게 된다.
# 이럴 때는 마크다운을 txt로 변환한 다음에 로딩을 해서 스플릿을 해줘야 한다.
# 먼저 필요한 패키지를 설치한다.

%pip install -q markdown html2text beautifulsoup4


In [ ]:
import markdown
from bs4 import BeautifulSoup

#Read the markdown file
text_path = './documents/income_tax.txt'
with open(markdown_path, 'r', encoding='utf-8') as md_file:
    md_content = md_file.read()

#Convert markdown to HTML
html_content = markdown.markdown(md_content)

#use beautifulsoup to extract the text from the HTML
soup = BeautifulSoup(html_content, 'html.parser')
text_content = soup.get_text()

#save the text to a .txt file
with open(text_path, 'w', encoding='utf-8') as txt_file:
    txt_file.write(text_content)

print("Markdown converted to plain text successfully!")




In [15]:
# 5.

# 이제 이 텍스트를 로딩해서 스플릿을 해준다.
# langchain_community 라이브러리에 있는 TextLoader를 사용한다.
from langchain_community.document_loaders import TextLoader

loader = TextLoader(text_path)
document_list = loader.load_and_split(text_splitter)

In [ ]:
document_list[47]

In [ ]:
# 6.

#예쁘게 표가 잘 나온다.
#크로마 DB를 만들어서 넣어준다.

%pip install -q langchain_chroma


In [ ]:
%pip install -qU langchain-openai
%pip install -qU langchain-huggingface

In [ ]:
# Embedding이 필요하다.

# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
# Embedding 설정
# from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings

# ✅ LangChain과 호환 가능한 Embeddings 설정
model_name = "nlpai-lab/KURE-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)





In [20]:
# 벡터 스토어를 만들어준다.

from langchain_chroma import Chroma

# ✅ 벡터 스토어 생성
vector_store = Chroma.from_documents(
    documents=document_list,  # 벡터화할 문서 리스트
    embedding=embeddings_function,  # ✅ SentenceTransformer 대신 사용
    collection_name="income_tax_collection",
    persist_directory="./income_tax_collection"  # 벡터 스토어 저장 경로
)

In [ ]:
# 이렇게 생긴 벡터 스토어를 기반으로 retrieval을 만들어 준다.

# ✅ Retrieval 객체 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

print("✅ 새로운 벡터 스토어 생성 완료!")

In [22]:
# 이제 질문을 해보자

query = "연봉 5천만원 직장인의 소득세는?"

In [ ]:
retriever.invoke(query)

In [24]:
# 7.

# 다시 리마인드를 하면 (1)start -> (2)retrieval -> (3)genarate_answer(답변 생성) -> (4)end(return) 이렇게 된다.
# state를 만들고 시작해보자.

from typing_extensions import List, TypedDict

class AgentState(TypedDict):
    query: str # 질문
    context: List[Document] # 컨텍스트(답변할 때 참고할 문서들: langchain의 Document 타입) - 경로는 3.1의 from langchain_core.documents import Document
    answer: str # 답변

In [25]:
# 그래프를 만들어 보자.
# 그래프 빌더를 만든다.

from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)

In [26]:
# 노드를 만든다. 두 가지가 필요하다. 
# 1. 문서를 가져오는 retrieve : 노드는 함수고 때문에 state를 인자로 받는다. 
# 2. 답변을 생성하는 generate

def retrieve(state: AgentState):
    query = state['query'] # 사용자의 질문을 꺼내온다.
    docs = retriever.invoke(query) # 질문을 활용해서 벡터스토어 우리가 만든 리트리버에 대해 검색을 한다.
    return {'context': docs} # 검색한 문서를 state에 넣어준다. state에 보면 context가 있는 것을 확인할 수 있다. 따라서 context에 docs를 담아주는 것.

In [27]:
# 이제 생성하는 generate를 만들어보자. 
#  - 기존에는 LLM에서 인보크만 했었다. 그래서 사용자 질문만 넣었다.
#  - 이번에는 사용자 질문과 컨텍스트를 넣어줘야 한다.
#  - rag에 효율적인 프롬프트를 작성해야 한다.
#  - 프롬프트는 직접 작성하는 것 보다 langsmith에서 주는 프롬프트를 가져다 쓰는게 좋다.
#  - https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=bd10098a-4810-4d31-9769-14cc090553ec

# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
prompt = hub.pull("rlm/rag-prompt")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")


In [28]:
# context 랑 query를 잘 받아와서, LLM을 invoke 하지 않고, 선언한 프롬프트를 활용해야한다.
# langchain의 LCEL 문법: 파이프 활용 해서 연결하는 것. 기반으로 체인을 만들어 준다.

def generate(state: AgentState):
    query = state['query']
    context = state['context']
    rag_chain = prompt | llm
    response = rag_chain.invoke({"question": query, "context": context})
    return {'answer': response}





In [ ]:
# # 연결을 시켜줘야 한다.
#  - start -> retrieve -> generate -> end
#  - (1) 노드 추가
#  - (2) 엣지 추가

# (1)노드 추가
graph_builder.add_node('retrieve', retrieve)
graph_builder.add_node('generate', generate)



In [ ]:

# START, END 임포트

from langgraph.graph import START, END

graph_builder.add_edge(START, 'retrieve')
graph_builder.add_edge('retrieve', 'generate')
graph_builder.add_edge('generate', END)


In [31]:
# 컴파일

graph = graph_builder.compile()

In [ ]:
# 그래프 출력

from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))




In [33]:
# ※ 이렇게 시퀀스 형식인 경우에는 노드와 엣지를 일일이 그리지 않고 더 간단한 방식으로도 그래프를 그릴 수 있다.

sequence_graph_builder = StateGraph(AgentState).add_sequence([retrieve, generate])

# 이렇게 하면 StateGraph에 이미 위와 같은 시퀀스가 들어간 상태이다. 
# 이렇게 한 후 엣지를 한 번 추가해준다.

In [ ]:
sequence_graph_builder.add_edge(START, 'retrieve') # START만 넣고 retrieve, generate는 빠진다.
# sequence_graph_builder.add_edge('retrieve', 'generate')
sequence_graph_builder.add_edge('generate', END)





In [35]:
# 이렇게 한 후 빌드를 한 번 해보면 그래프가 그려진다.

sequence_graph = sequence_graph_builder.compile()





In [ ]:
display(Image(sequence_graph.get_graph().draw_mermaid_png()))

## 코드가 훨씬 간단해졌다.

In [ ]:
## 이제 함수를 호출해 보자

initial_state = {'query': query}

graph.invoke(initial_state)

# 제공된 문서에는 연봉 5천만원 직장인 소득세에 대한 정보가 없다고 나온다.
# 왜냐하면 사용자의 질문이 효율이 떨어지기 때문이다.
# 이럴 때는 프롬프트를 수정해야 한다.

# 다음 시간에 retrieve를 하고 바로 generate를 해서 답변을 생성해서 답변을 리턴하는 것이 아니라 
# 검증을 한번 하고 문서가 사용자의 질문과 관련이 있으면 genenrate를 하고 그렇지 않으면 rewrite를 통해서 사용자의 질문을 문맥에 맡게 수정한 다음에 문서를 다시 가져오는 절차를 짚어보도록 하겠다.